# AI Agent を使った分析
- 自動車保険の約款データを使ったRAGの構築
- Text-to-SQLで自動車保険ビジネスにおける様々な部門の課題解決に直結するインサイトの提供

![img](https://lh3.googleusercontent.com/pw/AP1GczP0C8liJXnsqhXFB6JJ5ekY1n9o5yZds1KEFr9aVVM151oaEZDf9M13J-NnqlV3RT6YdApfpGiLDxJVrQzGR5O1mZxwwlzYPrhO_4MBDWz2f-3zFcLGxUU5KU_8Qr4C8Vv1bFCRZ6UenQibcp9MvrDe=w960-h540-s-no-gm?authuser=0)

## 1. ドライバー保険の約款データを使ったRAGの構築

![img](https://lh3.googleusercontent.com/pw/AP1GczOt0nmRojcDHtvpF8bjBXlpPXwrG0Z6PucmBr_chM_FL1J4sk71PHCe-gRSJ4lyOi142OWep7fdf7r0N0y-kqT181fz-WJc3Rc2ZAPMdGNNOVqc8MLveyRbLgw0pr2dFjC_9mNxkXciB5S8T8FfiiB6=w960-h540-s-no-gm?authuser=0)

### 1-1. AI_PARSE_DOCUMENT を使った PDF データの抽出
![img](https://lh3.googleusercontent.com/pw/AP1GczOKOjVq70egwB7rCuTetrHuGiIAL8fhNsXWqoLceZlUqOL_F9ZRKNw4slQXoJz5H00yK8e1RCtwaocc_XM6sB0Dazdb0MQk-d8Eqgouk0rmbEFB9cYP7alGClCEeRWFdWYAu39da9IgBJp3v490K-IX=w960-h540-s-no-gm?authuser=0)

![img](https://lh3.googleusercontent.com/pw/AP1GczN4ZRViZzd8ER-Qmwibk82f3Hjxm0ahtC7rqjDA6XLprtw0WctVLUNGr2bpB-HmicBAU5oZ-ESQTwL8eiB4hP_LTALHwByxTbcBwjUhd2MngVpkxi4uuieln6PxGF6YmA7V2bSMbEFwd-FjqN5NG9WJ=w960-h540-s-no-gm?authuser=0)
#### ２つの抽出モード
- **LAYOUT モード**
    - ほとんどのユースケース, 特に複雑な文書において推奨される
    - テキストや表などのレイアウト要素の抽出に特化して最適化されているため, ナレッジベースの構築 / 検索システムの最適化 / AIベースのアプリケーションの強化に向いている

- **OCR モード**
    - マニュアル、契約書等からの迅速かつ高品質なテキスト抽出に向いている


[ドキュメント](https://docs.snowflake.com/en/user-guide/snowflake-cortex/parse-document) 



In [ ]:
CREATE OR REPLACE TABLE handson.car_insurance.parsed_terms_and_conditions AS -- 新しいテーブルを作成または置き換えます
SELECT
    relative_path, -- ステージ内のファイルの相対パス（ファイル名を含む）を取得
    BUILD_STAGE_FILE_URL('@handson.car_insurance.terms_and_conditions', relative_path) as file_url,
    -- ステージ内のファイルにアクセスするためのURLを生成
    TO_FILE(BUILD_STAGE_FILE_URL('@handson.car_insurance.terms_and_conditions', relative_path) ) file_object,
    -- 生成したURLからファイルオブジェクトを作成 (AI関数に渡す準備)
    AI_PARSE_DOCUMENT(
        -- PDFなどのドキュメントを解析し、内容を抽出するSnowflakeのAI関数
        TO_FILE('@handson.car_insurance.terms_and_conditions', relative_path),
        -- 解析するファイルを指定 (ステージと相対パスを使用)
        {'mode':'LAYOUT', 'page_split': false}
        -- 解析オプションを設定
        -- 'mode':'LAYOUT' は、元のレイアウトを保持しつつ構造化されたテキストを抽出
        -- 'page_split': false は、ドキュメント全体を一つのコンテンツとして抽出 (ページごとには分割しない)
        ):content::string AS content
        -- AI_PARSE_DOCUMENTの出力（JSON形式）から 'content' キーの値（抽出されたテキスト）を取り出し、文字列型として 'content' 列に格納
FROM DIRECTORY(@handson.car_insurance.terms_and_conditions)
-- ファイル情報（relative_pathなど）を取得するために、指定されたステージ（ディレクトリテーブルとして機能）を参照
WHERE relative_path ILIKE '%.pdf';
-- 処理対象をファイルパスが '.pdf' で終わるファイルのみに限定（PDFファイルのみを対象とする）

In [ ]:
SELECT * FROM handson.car_insurance.parsed_terms_and_conditions;

In [ ]:
SELECT
    relative_path,  -- ファイルが保存されている場所（相対パス）
    file_url,       -- ファイルにアクセスするためのURL
    REGEXP_SUBSTR(relative_path, '[^/]+$') as title, -- relative_pathからファイル名（タイトル）を抽出
                                                     -- '[^/]+$' は「最後のスラッシュ（/）以降の全て」を意味
    content         -- ファイルの中身（内容）を取得
FROM
    handson.car_insurance.parsed_terms_and_conditions

### 1-2. SPLIT_TEXT_RECURSIVE_CHARACTER を使ったチャンクの作成

![img](https://lh3.googleusercontent.com/pw/AP1GczM-CD9_MvwVc3XtQUOFuoQyXLURyXfcp3OGWLmzZa6nERMBwyDy9qB_M557c7CqeYqpAuyDR2jWoIT7yIYhUqLUaueiOY6kLgp0ogU5UHmIy1kzHC7ng7WptxA70kijF5881nU4jd0CZ1l9NYm014v-=w960-h540-s-no-gm?authuser=0)

![img](https://lh3.googleusercontent.com/pw/AP1GczMa92b98auU3QY9eH_s2DdT0r21-1fL1MjtjUP-oDj3cMDA4h_nc0DiDaKejRW2RB3dDyJJSVpt1uiVrmBMXBUe6ikosOSWgVL6VJuPxD3wWIikfbieIB0nqRyvdyun5Vfey7wli1QdaOhxK-tL9mZc=w960-h540-s-no-gm?authuser=0)

本ハンズオンでは以下を設定
- 分割対象: contentカラムのテキストを分割
- フォーマット: 'markdown' - ヘッダー、コードブロック、テーブルで分割
- チャンクサイズ: 2500文字 - 各チャンクの最大文字数
- 重複サイズ: 500文字 - 隣接するチャンク間で重複させる文字数
- 処理結果: 2500文字以下のチャンクに分割され、500文字ずつ重複

[ドキュメント](https://docs.snowflake.com/en/sql-reference/functions/split_text_recursive_character-snowflake-cortex) (最新の情報は英語ドキュメントを参照)

In [ ]:
-- チャンク分割用の新しいテーブルの作成
CREATE OR REPLACE TABLE handson.car_insurance.parsed_terms_and_conditions_chunks (
    relative_path VARCHAR,   -- 元のファイルの場所(パス)
    file_url VARCHAR,        -- 元のファイルへのURL
    title VARCHAR,           -- 元のファイルのタイトル(ファイル名)
    CHUNK VARCHAR            -- 分割されたテキストの塊（チャンク）
);

-- 元のテーブルからデータを取得してチャンク分割
INSERT INTO handson.car_insurance.parsed_terms_and_conditions_chunks (relative_path, file_url, title, CHUNK)
SELECT
    relative_path,  -- ファイルのパス
    file_url,       -- ファイルのURL
    REGEXP_SUBSTR(relative_path, '[^/]+$') as title,    -- パスからファイル名だけを抽出
    c.value AS CHUNK    -- 分割されたテキストの塊（チャンク）
FROM
    handson.car_insurance.parsed_terms_and_conditions,
    -- 次の行から、テキスト分割処理を開始
    LATERAL FLATTEN( input => SNOWFLAKE.CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTER (
    content,      -- 分割したいテキスト
    'markdown',   -- テキストの形式を指定
    2500,         -- チャンクの最大サイズ(2500文字)
    500           -- チャンク間の重複部分のサイズ（500文字）
)) c;

In [ ]:
-- チャンクデータの確認
SELECT * FROM handson.car_insurance.parsed_terms_and_conditions_chunks;

### 1-3. Cortex Search を利用しコンテキストを理解可能なサービスを作成

![img](https://lh3.googleusercontent.com/pw/AP1GczOWEtOg4Zoz8bUiqaNLbQ3QnN88dX14O7lS6qAqY46OxoeuMMkdf0DpNRHzB9JDNga3DuSJKWF44armW91SORsSSbYN3bQGGsk-cPyBaCAuJ9x6UPRBrHgYSNQYO1NSm5LLREkr8MwjB4ijqD3Plv_Z=w960-h540-s-no-gm?authuser=0)

![img](https://lh3.googleusercontent.com/pw/AP1GczNb7-sVZOoYPnzVvmvx6SzsOUKi0IrxEJxkj702g4JIM4fYz7R_XpdOrS3mse6wKMZ8oRcljRT6YgtC0cwNF-qhoBSL2bzUXoE-qnc3xHiTP6913Wt5BlpkFiwpswaMsuRGJ8J6YlsLMimnT6KPb_9Z=w960-h540-s-no-gm?authuser=0)


In [ ]:
CREATE OR REPLACE CORTEX SEARCH SERVICE handson.car_insurance.terms_and_conditions_service
    ON chunk
    ATTRIBUTES relative_path, file_url, title
    WAREHOUSE = compute_wh
    TARGET_LAG = '30 day'
    EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
    AS (
        SELECT
            relative_path,
            file_url,
            title,
            chunk
        FROM handson.car_insurance.parsed_terms_and_conditions_chunks
    );

補足）Cortex Search のチューニング例
- 属性で絞る → filter
- 本文の“意味”で探す → query（セマンティック検索）
- 人気/新しさ等を反映 → scoring_config（numeric_boosts / time_decays / reranker）


REST API の例
```
POST https://<account_url>/api/v2/databases/<db>/schemas/<schema>/cortex-search-services/<service>:query
Content-Type: application/json
Authorization: Bearer <token>

{
  "query": "事故 対応",
  "columns": ["DOCUMENT_CONTENTS", "REGION", "CATEGORY", "LIKES", "CREATED_AT"],
  "filter": {
    "@and": [
      { "@eq": { "REGION": "JP" } },
      { "@or": [
        { "@contains": { "TAGS": "insight" } },
        { "@eq": { "CATEGORY": "guide" } }
      ]},
      { "@gte": { "CREATED_AT": "2025-07-01+09:00" } }
    ]
  },
  "scoring_config": {
    "functions": {
      "time_decays": [
        { "column": "CREATED_AT", "weight": 1.0, "limit_hours": 240 }
      ]
    }
  },
  "limit": 50
}
```


In [ ]:
-- アカウント内で作成された Cortex Search サービスを確認
SHOW CORTEX SEARCH SERVICES IN ACCOUNT;

SNOWFLAKE.CORTEX.SEARCH_PREVIEW 関数で単発クエリの結果をプレビューする
- ワークシートや Snowflake ノートブックから 単発クエリの結果をプレビューする用途の関数

In [ ]:
SELECT PARSE_JSON(
  SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
      'handson.car_insurance.terms_and_conditions_service',
      '{
         "query": "事故が発生した場合の対応手順を教えてください。",
         "columns": ["chunk", "title", "relative_path", "file_url"],
         "limit": 5
      }'
  )
)['results'] as results;

In [ ]:
SELECT 
    '事故対応手順検索' as search_type,
    (results.index + 1) as result_rank,
    results.value['title']::string as document_title,
    results.value['chunk']::string as content,
    LENGTH(results.value['chunk']::string) as content_length,
    -- results.value['file_url']::string as source_file_url
FROM (
    SELECT 
        PARSE_JSON(
            SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
                'handson.car_insurance.terms_and_conditions_service',
                '{
                    "query": "事故が発生した場合の対応手順を教えてください。",
                    "columns": ["chunk", "title", "file_url"],
                    "limit": 5
                }'
            )
        )['results'] as results_array
),
LATERAL FLATTEN(input => results_array) as results
ORDER BY results.index;

### 補足) Cortex Search ダウンロードリンク生成ツール
- Cortex Searchで検索したドキュメントのダウンロードリンクを生成するストアドプロシージャを用意
- 最終的にSnowflake Intelligenceを使ってソースとなるドキュメントのダウンロードが可能に

In [ ]:
-- ストアドプロシージャの定義（作成または置換）
CREATE OR REPLACE PROCEDURE get_file_presigned_url_sp( 
    RELATIVE_FILE_PATH STRING,                       -- 引数1：ファイルへの相対パス
    EXPIRATION_MINS INTEGER DEFAULT 60               -- 引数2：URLの有効期限(分), デフォルトは60分
)
RETURNS STRING                                       -- 戻り値：生成されたPresigned URL
LANGUAGE SQL                                         -- 使用言語：SQL
COMMENT = '@handson.car_insurance.terms_and_conditions内のファイルに対するpresigned URLを生成します。入力は相対ファイルパスです。'
EXECUTE AS CALLER                                    -- 実行権限：プロシージャを呼び出したユーザーの権限で実行

AS
$$
DECLARE                                              -- 変数宣言の開始
    presigned_url STRING;                            -- 生成されたURLを格納する変数
    sql_stmt STRING;                                 -- 実行するSQL文を格納する変数
    expiration_seconds INTEGER;                      -- 有効期限(秒)を格納する変数
    stage_name STRING DEFAULT '@handson.car_insurance.terms_and_conditions'; -- ファイルが格納されているステージ名
BEGIN                                                -- プロシージャ本体の処理開始
    expiration_seconds := EXPIRATION_MINS * 60;      -- 有効期限を「分」から「秒」に変換
    
    sql_stmt := 'SELECT GET_PRESIGNED_URL(' || stage_name || ', ' || '''' || RELATIVE_FILE_PATH || '''' || ', ' || expiration_seconds || ') AS url';
                                                     -- Presigned URLを生成するためのSQL文を組み立て

    EXECUTE IMMEDIATE :sql_stmt;                     -- 組み立てたSQL文を実行

    SELECT "URL"
    INTO :presigned_url                              -- 実行結果の「URL」列をpresigned_url変数に格納
    FROM TABLE(RESULT_SCAN(LAST_QUERY_ID()));        -- 最後のクエリ（GET_PRESIGNED_URL）の結果をスキャン

    RETURN :presigned_url;                           -- 生成されたPresigned URLを戻り値として返す
END;
$$;

In [ ]:
CALL get_file_presigned_url_sp('yakkan_driver_insurance.pdf', 60)

### 1-4. Streamlitでお試し

![img](https://lh3.googleusercontent.com/pw/AP1GczOXF3HrG1b90sR-fNbY_up0KN6zpzmtkjxbXVMvDZ-0IXy1vmC0wefgFVdpmBTG7b__wUy_y7RnXtS9C8keWIzjxuJt6-8oVwnJ6bAG7IVqUCE8ncaaWRu97_k-jkSDbZ0vuYrNMIHMYfiS5TmEqAhZ=w960-h540-s-no-gm?authuser=0)

## Challenge Mode 🏂
 - かんぽ生命様のドキュメントでトライしてみましょう
 - S3のパス：s3://snowflake-workshop-lab/japan/handson-jpi/PDFs


## 2. Text-to-SQLで自動車保険ビジネスにおける様々な部門の課題解決に直結するインサイトの提供
![img](https://lh3.googleusercontent.com/pw/AP1GczNxo-PTxXi6JXI9Jybc565_rvOwrosABKsOrDYvY-BbN5PAY2ZtJQwDSKqmvCUCNWNAzAvwHlPWYGxoWB54AbTaMYL4vm0y3d4sZSbObawYM5TBfy5jMqZTEcfAUEh67ChxnmvAOSh06Dae6Z98LSQn=w960-h540-s-no-gm?authuser=0)

### 2-1. セマンティックビューの作成
![img](https://lh3.googleusercontent.com/pw/AP1GczMS8gfqMW9e7b-0Anl8baXt6yvubLJ6yzijJh49U4-l6eLjNpJnvTKjXKcIc2iNRlkyzV0ZEPogKS9DuvqSXEUTOmRunMIo6g2UVOfgveYWKj1FHDbfk3vVVvWEOTy6w32Kld79EfpIDRhGkkAd5PR0=w960-h540-s-no-gm?authuser=0)
1. GUI でセマンティックビューを作成してみる
    - 「AIとML」→「Cortex分析」→「新規作成」→「Create new Semantic View」を選択
2. 次のセルを実行しSQLでセマンティックビューを作成し、GUIで動作を確認する



![img](https://lh3.googleusercontent.com/pw/AP1GczOgagfCPfNAZLjeHdon2_SwCuZqwm1kL-Bwo0LKjjranKs-rlvqT-y4vZC4GXttZEv_SxRHvd3kNjZT_9qy5UZTv8SAfiXFXU-aN41q618ZT37sHRk0kF1FIi82H0guENJuOV3QH3L5Ax8T3353fzCq=w960-h540-s-no-gm?authuser=0)

#### 補足) セマンティックビューの定義について

![img](https://lh3.googleusercontent.com/pw/AP1GczMNPGvZ9ie_XnVUiT88JnEMKI1wtr12kgj_5cs-vOuHfj9Y1WIBIS2XAFZC_B2UxKrSoGJnV-7ljYveRE-m5zcZpZ2OtoGMgjxBvqi2CHaD-uGfR9Pql_g2OHc0jXJZh0vRgDdITDQwYiZFcXsFzvaK=w960-h540-s-no-gm?authuser=0)

**ディメンション** ： 
- データをグループ化したり、絞り込んだり、分析したりするための「切り口」や「視点」を提供するもの
- 例) 購入日、顧客の詳細、製品カテゴリー、場所など、「誰が」「何を」「どこで」「いつ」問題に答えるもの

**ファクト** ： 
- ファクトは分析の最終的な指標そのものではなく、その指標を計算するための基礎となる構成要素

**メトリクス** ： 
- 複数の行にわたって同じテーブルのファクトや他の列を（SUM、AVG、COUNT などの関数を使用して）集約することによって計算される、ビジネス・パフォーマンスの定量化可能な尺度
- 例) 総収入や利益率など。メトリクスは、ビジネスの意思決定を促進するレポートやダッシュボードの KPI を表します。



In [ ]:
CREATE OR REPLACE SEMANTIC VIEW INSURANCE_CLAIMS_SEMANTIC_VIEW
    tables (
        CLAIMS as CLAIMS primary key (CLAIM_ID) 
            with synonyms=('保険金請求','請求','事故') 
            comment='保険金請求に関する中心的なファクトテーブル',
        POLICIES as POLICIES primary key (POLICY_ID) 
            with synonyms=('保険契約','契約','ポリシー') 
            comment='保険契約者と車両に関するディメンションテーブル',
        REPAIR_SHOPS as REPAIR_SHOPS primary key (SHOP_ID) 
            with synonyms=('修理工場','工場','パートナー工場') 
            comment='提携している修理工場に関するディメンションテーブル',
        REPAIR_ORDERS as REPAIR_ORDERS primary key (ORDER_ID) 
            with synonyms=('修理指示','修理明細','作業指示') 
            comment='各請求に対する修理作業の明細データ'
    )
    relationships (
        CLAIMS_TO_POLICIES as CLAIMS(POLICY_ID) references POLICIES(POLICY_ID),
        CLAIMS_TO_SHOPS as CLAIMS(REPAIR_SHOP_ID) references REPAIR_SHOPS(SHOP_ID),
        ORDERS_TO_CLAIMS as REPAIR_ORDERS(CLAIM_ID) references CLAIMS(CLAIM_ID)
    )
    facts (
        CLAIMS.claim_record as 1 
            comment='請求レコード数。件数のカウントに使用',
        CLAIMS.approved_cost_yen as APPROVED_COST_YEN 
            comment='承認された修理費用（円）',
        CLAIMS.estimate_cost_yen as ESTIMATE_COST_YEN 
            comment='見積もり修理費用（円）',
        CLAIMS.paid_to_date_yen as PAID_TO_DATE_YEN 
            comment='支払済み金額（円）',
        POLICIES.policy_record as 1 
            comment='契約レコード数。件数のカウントに使用',
        POLICIES.annual_premium_yen as ANNUAL_PREMIUM_YEN 
            comment='年間保険料（円）',
        REPAIR_ORDERS.repair_cost as LINE_TOTAL_AFTER_ADJ_YEN
            comment='修理明細の合計金額（調整後、円）',
        REPAIR_ORDERS.labor_hours as LABOR_HOURS
            comment='作業時間（時間）'
    )
    dimensions (
        -- CLAIMS テーブルのディメンション
        CLAIMS.incident_date as INCIDENT_DATE 
            with synonyms=('事故日','発生日') 
            comment='事故が発生した日付',
        CLAIMS.incident_year as YEAR(incident_date) 
            with synonyms=('事故年','発生年') 
            comment='事故が発生した年',
        CLAIMS.incident_month as MONTH(incident_date) 
            with synonyms=('事故月','発生月') 
            comment='事故が発生した月',
        CLAIMS.claim_status as CLAIM_STATUS 
            with synonyms=('請求ステータス','ステータス') 
            comment='保険金請求の現在のステータス (Open, Approved, Paid, Denied)',
        CLAIMS.loss_cause as LOSS_CAUSE 
            with synonyms=('事故原因','原因') 
            comment='損害の原因 (Collision, Weather, Theftなど)',
        CLAIMS.severity as SEVERITY 
            with synonyms=('損害の大きさ','深刻度') 
            comment='損害の深刻度 (Minor, Moderate, Severe, Total Loss)',
        CLAIMS.incident_prefecture as INCIDENT_PREFECTURE 
            with synonyms=('事故発生都道府県','事故現場の都道府県') 
            comment='事故が発生した都道府県',
        CLAIMS.fraud_score as FRAUD_SCORE
            with synonyms=('不正スコア', '詐欺スコア')
            comment='請求の不正行為の可能性を示すスコア',
        -- POLICIES テーブルのディメンション
        POLICIES.vehicle_make as VEHICLE_MAKE 
            with synonyms=('自動車メーカー','メーカー') 
            comment='車両のメーカー',
        POLICIES.vehicle_model as VEHICLE_MODEL
            with synonyms=('車種','モデル')
            comment='車両のモデル名',
        POLICIES.vehicle_age_band as VEHICLE_AGE_BAND 
            with synonyms=('車齢帯','車両年式') 
            comment='車両の年式帯 (Newer, Mid, Older)',
        POLICIES.vehicle_use as VEHICLE_USE
            with synonyms=('車両用途','使用目的')
            comment='車両の主な使用目的 (Personal, Commute, Commercial)',
        POLICIES.driver_age as DRIVER_AGE
            with synonyms=('運転者年齢','ドライバー年齢')
            comment='契約時の運転者の年齢',
        POLICIES.policy_status as POLICY_STATUS 
            with synonyms=('契約ステータス') 
            comment='保険契約のステータス (Active, Lapsed, Cancelled)',
        POLICIES.registered_prefecture as REGISTERED_PREF
            with synonyms=('登録都道府県')
            comment='車両が登録されている都道府県',
        -- REPAIR_SHOPS テーブルのディメンション
        REPAIR_SHOPS.shop_name as SHOP_NAME 
            with synonyms=('修理工場名','工場名') 
            comment='修理工場の名称',
        REPAIR_SHOPS.shop_prefecture as PREFECTURE 
            with synonyms=('工場都道府県','工場所在地') 
            comment='修理工場の都道府県',
        REPAIR_SHOPS.partner_tier as PARTNER_TIER 
            with synonyms=('パートナーランク','ティア') 
            comment='修理工場のパートナーランク (A, B, C)',
        REPAIR_SHOPS.shop_rating as RATING
            with synonyms=('工場評価','評価')
            comment='修理工場の評価スコア',
        -- REPAIR_ORDERS テーブルのディメンション
        REPAIR_ORDERS.work_start_date as WORK_START_DATE
            with synonyms=('修理開始日')
            comment='修理作業の開始日',
        REPAIR_ORDERS.work_end_date as WORK_END_DATE
            with synonyms=('修理完了日')
            comment='修理作業の完了日'
    )
    metrics (
        CLAIMS.TOTAL_CLAIMS as COUNT(claims.claim_record) 
            with synonyms=('総請求件数','合計請求件数')
            comment='すべての保険金請求の合計件数',
        CLAIMS.TOTAL_APPROVED_COST as SUM(claims.approved_cost_yen) 
            with synonyms=('総承認費用','合計承認費用')
            comment='承認された修理費用の合計額',
        CLAIMS.AVERAGE_APPROVED_COST as AVG(claims.approved_cost_yen) 
            with synonyms=('平均承認費用')
            comment='承認された修理費用の平均額',
        CLAIMS.TOTAL_PAID_AMOUNT as SUM(claims.paid_to_date_yen)
            with synonyms=('総支払額','合計支払額')
            comment='支払われた保険金の合計額',
        POLICIES.TOTAL_POLICIES as COUNT(policies.policy_record) 
            with synonyms=('総契約件数','合計契約件数')
            comment='保険契約の総件数',
        POLICIES.TOTAL_PREMIUM_REVENUE as SUM(policies.annual_premium_yen) 
            with synonyms=('総保険料収入','合計保険料')
            comment='年間保険料の合計額',
        REPAIR_ORDERS.TOTAL_REPAIR_COST as SUM(repair_orders.repair_cost)
            with synonyms=('総修理費用','合計修理費用')
            comment='修理にかかった費用の総額'
    )
    COMMENT='自動車保険の契約、請求、修理に関する分析のためのセマンティックビュー';

参考）セマンティックモデル/ビューで改善ポイントまとめ

| ポイント                           | 概要                                                   |
| ---------------------------- | ----------------------------------------------------- |
| シンプルに始める                     | 範囲を絞り、少数のテーブルと列から開始し、段階的に拡張                           |
| ユースケースを明確に定義する               | 各セマンティックモデルは特定のビジネス領域に特化させる                           |
| 精度向上のためのテーブルと列の制限            | ユーザーが尋ねる可能性のある主要な質問に必要なテーブルと列のみを含める                   |
| Text to SQL で利用するテーブルのセットアップ | スタースキーマをサポートし、データの重複を避けることで LLM の判断を補助                |
| カラムの選択                       | 類似した名前や意味を持つ列を避け、モデルの混乱を防ぐ                            |
| メトリクスを個別の列に分割                | 各指標を個別の列にフラット化し、より多くの意味的情報を提供する                       |
| 複雑な計算を直接定義する                 | メジャー、ディメンション、時間、フィルターをセマンティックモデル内で定義する                |
| パフォーマンスの改善                   | まずは低コストで効果的な手法から着手し、徐々に複雑な機能強化を検討する                   |
| シノニムと説明書きについて                | 論理列名を調整し、企業固有の用語や慣用表現、同義語を追加する                        |
| 半構造化データについて                  | 可能であれば前処理を行い、個別の列にフラット化を推奨                            |
| サンプル値                        | 各ディメンションにサンプル値を提供することで、LLM が正確なクエリを作成するのを補助           |
| Cortex Search との統合           | 大規模な値の集合に対するフィルタリングに Cortex Search の使用を推奨             |
| Cortex Search の設定            | 上記を利用する場合はセマンティックモデルに組み込む前に、Cortex Search を別途設定する必要あり |
| Verified query repository    | 検証済み SQL をセマンティックモデルに追加し、複雑なクエリや一般的な質問に利用する             |
| カスタムインストラクション                | LLM の挙動を調整するために、明確で具体的な指示を記述                          |
| 質問のカテゴリライズ                   | SQL 生成を試みる前に質問分類を定義し、関連性のない質問は極力避ける                      |
| SQL 生成                       | SQL の生成方法を指定し、フォーマットやフィルターを適用する                       |
| カスタムインストラクション                | 非英語圏の言語（例：日本語）の文字列を翻訳しないよう指示が可能                       |
| ライフサイクル                      | ユーザーフィードバックを収集し、セマンティックモデルを継続的に洗練させる                  |


### 2-2. Streamlitでお試し

![img](https://lh3.googleusercontent.com/pw/AP1GczOMWOxz_hM5Igo2KuKzCUm5tevM3dadU9g1Vr7OCl-ZaW87jZkHUwZVoRcxyJ8uiyK4DrY8pVkrrbHLy1U4eP_b0Iku6jo5AaS6vMto41C4eqfzpiuTnJIsVyzoRTRaqLn5v0QDh8bj312LlyaaC4r7=w960-h540-s-no-gm?authuser=0)


## Challenge Mode 🏂
*Easy Mode* 
 - Citibikeのデータ

*Hard Mode*
 - QuickStartsのデータ： [URL](https://quickstarts.snowflake.com/guide/getting_started_with_cortex_analyst/index.html)

### 3-1. 作成したオブジェクト(Cortex Analyst / Search)を使って Cortex Agent を作成する

![img](https://lh3.googleusercontent.com/pw/AP1GczOT9Wq8AXzsPOrqNWXaoiZt3CMthK2Sku200Fuoag2G0QTH4YJj1QUWav7C2JHBTQVo-AK5k6h6Njxwo1i0jbAq5g3k-sXAkL1CXa5TKUZpTgtwo4EdYJ-feScR1IraJhUPwCmOI7Ngn21hNWMfD2V1=w960-h540-s-no-gm?authuser=0)

- SQL で実施 (GUIでも可能)

In [ ]:
CREATE OR REPLACE AGENT snowflake_intelligence.agents.driver_insurance_chatbot_agent
WITH PROFILE='{ "display_name": "driver_insurance_agent" }'
    COMMENT=$$ 自動車保険に関する質問に回答するエージェントです。 $$
FROM SPECIFICATION $$
{
  "models": {
    "orchestration": ""
  },
  "instructions": {
    "response": "あなたは自動車保険に関するのデータマートにアクセスできるデータアナリストです。ユーザーが日付範囲を指定しない場合は、2025年と仮定してください。すべてのドメインからのデータを活用して、ユーザーの質問を分析し回答してください。可能であれば視覚化を提供してください。トレンドラインは線グラフをデフォルトとし、カテゴリは棒グラフを使用してください。",
    "orchestration": "既知のエンティティに対してcortex searchを使用し、詳細な分析のためにcortex analystに結果を渡してください。",
    "sample_questions": [
      {
        "question": "事故が発生した都道府県ごとに、平均承認費用を比較して、高い順に並べて可視化してください"
      }
    ]
  },
  "tools": [
    {
      "tool_spec": {
        "type": "cortex_analyst_text_to_sql",
        "name": "DRIVER INSURANCE SYSTEM DATAMART",
        "description": "お客様の自動車保険に関するデータをクエリすることをユーザーに許可します。"
      }
    },
    {
      "tool_spec": {
        "type": "cortex_search",
        "name": "DRIVER INSURANCE SEARCH",
        "description": "このツールはドライバー保険の約款データを検索するために使用します。"
      }
    },
    {
      "tool_spec": {
        "type": "generic",
        "name": "Doc URL Tool",
        "description": "このツールは、参照ドキュメント用のCortex Searchツールから取得したrelative_pathを使用し、ユーザーがドキュメントを表示・ダウンロードするための一時URLを返します。\n\n返されたURLは、ドキュメントタイトルをテキストとし、このツールの出力をURLとするHTMLハイパーリンクとして表示する必要があります。\n\nURLにPDFが含まれていないPDFドキュメントのURL形式は次のようになります。ファイルをダウンロードする代わりにブラウザでPDFドキュメントが開くようにハイパーリンク形式を作成してください。\nhttps://domain/path/unique_guid",
        "input_schema": {
          "type": "object",
          "properties": {
            "expiration_mins": {
              "description": "デフォルトは5にする必要があります",
              "type": "number"
            },
            "relative_file_path": {
              "description": "これはCortex Searchツールから取得されるrelative_pathの値です。",
              "type": "string"
            }
          },
          "required": [
            "expiration_mins",
            "relative_file_path"
          ]
        }
      }
    }
  ],
  "tool_resources": {
    "DRIVER INSURANCE SYSTEM DATAMART": {
      "semantic_view": "handson.car_insurance.insurance_claims_semantic_view"
    },
    "DRIVER INSURANCE SEARCH": {
      "id_column": "relative_path",
      "max_results": 5,
      "name": "handson.car_insurance.terms_and_conditions_service",
      "title_column": "title"
    },
    "Doc URL Tool": {
      "execution_environment": {
        "query_timeout": 0,
        "type": "warehouse",
        "warehouse": "COMPUTE_WH"
      },
      "identifier": "handson.car_insurance.get_file_presigned_url_sp",
      "name": "GET_FILE_PRESIGNED_URL_SP(VARCHAR, DEFAULT NUMBER)",
      "type": "procedure"
    },
  }
}
$$;

### 3-2. Snowflake Intelligence を使った AI Agent チャットボットの作成

![img](https://lh3.googleusercontent.com/pw/AP1GczMGwzIHe61li9c-8mNH5rxiQmSR3qOJ0mpAzVVfs3DOBvX7yzS7JbLJ7BOXbTEw9E5zGZat0DED-78NtHRzMgXOnqtSk5SVQHdF8DfQXQnLm53bmytRiM4-6t2ae2nicqLtSrjrH1Ur4Hw0JxKkAsMj=w960-h540-s-no-gm?authuser=0)
- GUIで作成

### 「AIML」→「Snowflakeインテリジェンス」をクリックしてSnowflakeインテリジェンスを起動

以下、Snowflakeインテリジェンスに対する質問のサンプル

---
Cortex Search向け質問

質問1：手続きについて
- 保険金の手続きを完了してから、いつまでに保険金を支払っていただけますか？尚、根拠となるソースをダウンロードできるようにしてください。

質問2：補償内容について
- 日常生活で、誤ってお店の商品を壊してしまいました。個人賠償責任補償特約で補償されますか？

質問3：保険金が支払われないケースについて
- 地震が原因でケガをした場合、傷害保険の保険金は支払われますか？

---
Cortex Analyst向け質問

質問1：地理的リスクとコストの分析
- 事故が発生した都道府県ごとに、平均承認費用を比較して、高い順に並べて可視化して

質問2：車両特性と事故原因の関連性分析
- 自動車メーカー別に、事故原因ごとの請求件数をクロス集計で見せてください。

質問3：提携工場のパフォーマンス評価
- 各修理工場の平均修理期間（修理完了日 - 修理開始日）を計算し、期間が長い順にトップ5の工場を教えてください。
